## import Library

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import *
from pyspark.sql.functions import *

from pyspark.ml.feature import VectorAssembler

from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

import matplotlib.pyplot as plt
import seaborn as sns

spark = SparkSession.builder \
                    .appName('RecommendModeling_ALS') \
                    .getOrCreate()

spark.conf.set("spark.sql.execution.arrow.enabled", "true")
import time
from datetime import datetime,timedelta
import pickle
start = time.time()

In [0]:
name ='config name' 
version = 'V1' # Config
modelversion = 'M1'
appid = '1111' # Config
datetime_obj = (datetime.now())
createdate = datetime_obj.strftime("%Y%m%d")

# Location
location = "recommendation/project/"  ## path = recommendation/{appname}/
dbutils.fs.mkdirs("/mnt/blob/%s" % (location))
print(location)

# Export the first one
datetime_obj = (datetime.now())
createdate = datetime_obj.strftime("%Y%m%d")

# Get Data

In [0]:
transaction_df = spark.sql(transaction_query)

#2. Vectorize

In [0]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

cat_col = ['categoryname',
           'campaigntypename',
           ]
# for i in cat_col:
#   outputCol = i +'_idx'
#   w = Window.orderBy(i)
#   transaction_df = transaction_df.withColumn(outputCol, dense_rank().over(w))


# แปลงค่าหมวดหมู่
indexers = [
    StringIndexer(inputCol=col, outputCol=col + "_idx").fit(transaction_df)
    for col in cat_col
]

# ใช้ StringIndexer

pipeline = Pipeline(stages=indexers)
transaction_df = pipeline.fit(transaction_df).transform(transaction_df)

In [0]:
transaction_df.printSchema()

In [0]:
transaction_pd = transaction_df.toPandas()

In [0]:
transaction_pd

#3. EDA

In [0]:
sns.set(rc={'figure.figsize':(25,10)})

In [0]:
pd.crosstab(transaction_pd['categoryname_cal'],columns='s').sort_values('s', ascending=False)

In [0]:
### Look Distribution by cat

pd.crosstab(transaction_pd['categoryname'],columns='s').sort_values('s', ascending=False)

###3.1 Filter Category UnUsed

In [0]:
transaction_pd[transaction_pd['categoryname_cal'].isin([......])]

In [0]:
transaction_pd2 = transaction_pd.copy()

In [0]:
transaction_pd2 = transaction_pd2[transaction_pd2['categoryname'] !='รับฟรี']
transaction_pd2 = transaction_pd2[transaction_pd2['campaigntypename'] !='Free']

In [0]:
transaction_pd2

In [0]:
transaction_pd2.isna().sum()

In [0]:
transaction_pd2 = transaction_pd2.drop(columns=[....])

#### plot distribution cross campaign and cat

In [0]:
display(transaction_pd2)

##3.1.2 Explore Number of count campaigns

In [0]:
transaction_pd2.groupby(['campaignid' ,'campaignname'],as_index=False)['refid'].nunique().sort_values(by='refid',ascending=False).describe()

In [0]:
cnt_campaignused  = transaction_pd2.groupby(['campaignid' ,'campaignname'],as_index=False)['id'].nunique().sort_values(by='id',ascending=False)

##3.2 Check price

In [0]:
transaction_pd2['price'] = transaction_pd2['price'].astype(int)
transaction_pd2['price_campaign'] = transaction_pd2['price_campaign'].astype(int)

In [0]:
pd.crosstab(transaction_pd2['price'],columns='s')

In [0]:
transaction_pd2[transaction_pd2['price'] == 0]

In [0]:
transaction_pd2 = transaction_pd2[transaction_pd2['price'] != 0]

In [0]:
display(transaction_pd2)

In [0]:
df = transaction_pd2.copy()
#df = df[['campaignid', 'campaignname', 'categoryname_cal','priceperunit_campaign']].rename(columns={'categoryname_cal':'category'})
df = df[['campaignid', 'campaignname', 'categoryname','priceperunit']].rename(columns={'categoryname':'category'})
df = df.drop_duplicates()

In [0]:
df = df.merge(cnt_campaignused[['campaignid','id']].rename(columns={'id':'campaigncount'}) ,on='campaignid',how='left')

In [0]:
df

#4. Clear Category
- Clean text category

In [0]:
import re
cat_un_process = df['category'].tolist()

cat_process = []
pattern1 = r'[&, " ", "  ", "   "]'
for i in cat_un_process:
  i = re.sub(pattern1, '', i)  
  print(i)
  cat_process.append(i)

df['cate_cleamed'] = cat_process

#6. Text Cleaning 
- Cleaning Tag in text use regex

In [0]:
# Raw name
name_un_process = df['campaignname'].tolist()

import re
name_process = []
#pattern1 = r'[0-9, ., :, ",]'
pattern2 = r'[-, _, \(, \), #, '', /, &, %,  ,+, '', °,]'
for i in name_un_process:
 # i = re.sub(pattern1, '', i)  
  i = re.sub(pattern2, '', i)
  name_process.append(i)


In [0]:
name_process

##6.1 Tokenize (Cut of Wording)
- เราจะทำการกรอง Wording ที่เป็นลักษณะเฉพาะ จนเกินไปออก
- ref https://medium.com/@witchapongdaroontham/%E0%B8%82%E0%B8%B1%E0%B9%89%E0%B8%99%E0%B8%95%E0%B8%AD%E0%B8%99%E0%B8%81%E0%B8%B2%E0%B8%A3%E0%B9%80%E0%B8%95%E0%B8%A3%E0%B8%B5%E0%B8%A2%E0%B8%A1%E0%B8%82%E0%B9%89%E0%B8%AD%E0%B8%A1%E0%B8%B9%E0%B8%A5%E0%B8%9B%E0%B8%A3%E0%B8%B0%E0%B9%80%E0%B8%A0%E0%B8%97-text-%E0%B8%A0%E0%B8%B2%E0%B8%A9%E0%B8%B2%E0%B9%84%E0%B8%97%E0%B8%A2-%E0%B9%81%E0%B8%9A%E0%B8%9A%E0%B8%87%E0%B9%88%E0%B8%B2%E0%B8%A2%E0%B9%86-%E0%B9%82%E0%B8%94%E0%B8%A2%E0%B9%83%E0%B8%8A%E0%B9%89-python-simple-thai-text-preprocessing-c8c46ca3ce46
- Using pythai NLP

In [0]:

high_freq_corpus = [................]

price_corpus = ['10 บาท', '20 บาท', '30 บาท', '5 บาท', '25 บาท','45 บาท','57 บาท','55 บาท','100 บาท','250 บาท','5','10','20','79','30']

BrandName_corpus = [.....................]
                 

In [0]:
from pythainlp.tokenize import word_tokenize
import numpy as np

final_token = []
for i in name_process:
  b = word_tokenize(i, engine='newmm')
  print(b)
  
  
  # Cut the bias words
  for i in range(len(b)):
    if b[i] in (high_freq_corpus):
      b[i] = ""
    #elif b[i] in (price_corpus):
    #  b[i] = ""
   # elif b[i] in (BrandName_corpus):
    #  b[i] = ""
 
  b = " ".join(b)
  #b = b + " " + str(df.iloc[i, 3])
  
  final_token.append(b)

In [0]:
final_token

#7. Copmbine in one DF

In [0]:
df_campaignname = df['campaignname'].tolist()

In [0]:

new_df = pd.DataFrame({
                        'name_index': id_,
                        'name': df_campaignname,
                        'name_cleaned': final_token
                      })
                     

In [0]:
'''
new_df = pd.DataFrame({
                        'name_index': id_,
                        'name': df_campaignname,
                        'name_cleaned': name_process
                      })
'''

Out[45]: "\nnew_df = pd.DataFrame({\n 'name_index': id_,\n 'name': df_campaignname,\n 'name_cleaned': name_process\n })\n"

In [0]:
#new_df['name_cleaned2'] = new_df['name_cleaned']+new_df['price'].astype(str)

In [0]:
new_df = new_df.reset_index(drop = True).reset_index()

In [0]:
new_df['name_cleaned'].unique()

#8 Feature processing
- Create Feature Using Tfid (term frequency-inverse document frequency) =  Concept คือ เป็นการหาจำนวนครั้งที่เกิดขึ้น ใน คำนั้นๆในแต่ละ ข้อความ ซึ่งคำไหนอยู่ใน text มากๆ ก็มีค่า tfid มากตาม
- ref : https://medium.com/@witchapongdaroontham/%E0%B8%82%E0%B8%B1%E0%B9%89%E0%B8%99%E0%B8%95%E0%B8%AD%E0%B8%99%E0%B8%81%E0%B8%B2%E0%B8%A3%E0%B9%80%E0%B8%95%E0%B8%A3%E0%B8%B5%E0%B8%A2%E0%B8%A1%E0%B8%82%E0%B9%89%E0%B8%AD%E0%B8%A1%E0%B8%B9%E0%B8%A5%E0%B8%9B%E0%B8%A3%E0%B8%B0%E0%B9%80%E0%B8%A0%E0%B8%97-text-%E0%B8%A0%E0%B8%B2%E0%B8%A9%E0%B8%B2%E0%B9%84%E0%B8%97%E0%B8%A2-%E0%B9%81%E0%B8%9A%E0%B8%9A%E0%B8%87%E0%B9%88%E0%B8%B2%E0%B8%A2%E0%B9%86-%E0%B9%82%E0%B8%94%E0%B8%A2%E0%B9%83%E0%B8%8A%E0%B9%89-python-simple-thai-text-preprocessing-c8c46ca3ce46

- https://medium.com/@cmukesh8688/tf-idf-vectorizer-scikit-learn-dbc0244a911a
- https://practicaldatascience.co.uk/data-science/how-to-create-content-recommendations-using-tf-idf

In [0]:
def identity_fun(text):
    return text

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel

#vector = TfidfVectorizer()
vector = TfidfVectorizer(analyzer = 'word', #this is default
                                   tokenizer=identity_fun, #does no extra tokenizing
                                   preprocessor=identity_fun, #no extra preprocessor
                                   token_pattern=None)
TFID = vector.fit_transform(new_df['name_cleaned'])
new_df['encoded_feature_TFID'] = TFID

#cosine_similarities = linear_kernel(TFID, TFID)

In [0]:
feature = TFID
label = new_df['name_index']

# 9. Train Model

In [0]:
import mlflow
import mlflow.pyfunc
import mlflow.sklearn
from sklearn.metrics import roc_auc_score
from mlflow.models.signature import infer_signature
mlflow.sklearn.autolog()

class SklearnModelWrapper(mlflow.pyfunc.PythonModel):
  def __init__(self, model):
    self.model = model
    
  def predict(self, context, model_input):
    return self.model.predict_proba(model_input)[:,1]
  
  

##9.1 RandomForest

In [0]:
from sklearn.ensemble import RandomForestClassifier
cls = RandomForestClassifier(criterion='entropy', n_estimators=750)

cls.fit(feature, label)

2022/10/18 06:27:05 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '34ac6e1ba676420eb8258f78e237ef3e', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
/databricks/python/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 _warn_prf(average, modifier, msg_start, len(result))
Out[54]:

RandomForestClassifier(criterion='entropy', n_estimators=750) In a Jupyter environment, please rerun this cell to show the HTML representation or trust the notebook. On GitHub, the HTML representation is unable to render, please try loading this page with nbviewer.org. RandomForestClassifier RandomForestClassifier(criterion='entropy', n_estimators=750)

In [0]:
def get_data(index, input_data, encoded):  
  chk_size = input_data[input_data['name_index'] == index]
  #print(chk_size)
  if chk_size.shape[0] == 0:
    data_index = 0
  else:
    data_index  = input_data[input_data['name_index'] == index].index[0]
    #print("data Index :",data_index)
    
  if encoded == 'TFID':
    data = TFID[data_index]
  elif encoded == 'COUNTVEC':
    data = COUNTVEC[data_index]
  return data, data_index

In [0]:
# Insert data
def get_TopN(index, data, model):
  # Name of prduct
  name = new_df[new_df['index'] == index].iloc[0, 2]
  # Master Campaign
  master_campaign_id = new_df[new_df['index'] == index].iloc[0, 1]
  #print('sss :' ,master_campaign_id)
  # Get Prob
  topN = model.predict_proba(data)
  # Sort uindex
  topN_index =  np.argsort(-topN, axis=1)[:,:10][0]
  #print('topN_index==>' , topN_index)
  # Rec Algolithm
  rows = []
  lst_prod_id = []
  lst_prod = []
  lst_campaignid  = []
  lst_prod_master = []
 # print('>>>>> Recommend:', name)
  x = 0
  while x <= 4:
    select_TopN = topN_index[x]
    rec_product_id = model.classes_[select_TopN]
   # print('>>>>> rec_product_id:', rec_product_id)
    rec_product = new_df[new_df['name_index'] == rec_product_id].iloc[0, 2]
    if rec_product == name:
      x = x
    else:
      rows.append(x)
      lst_campaignid.append(master_campaign_id)
      lst_prod_id.append(rec_product_id)
      lst_prod.append(rec_product)
      lst_prod_master.append(name)
      #print(x,'. %s'%(rec_product))
    x += 1
    
  return rows,lst_prod,lst_prod_id,lst_campaignid,lst_prod_master




In [0]:
lst_campaign = df['campaignid'].tolist()
#lst_campaign

In [0]:
df_row = pd.DataFrame() 
df_product = pd.DataFrame()
df_master_id = pd.DataFrame() 
df_product_id = pd.DataFrame() 
df_master_product = pd.DataFrame()
for i in lst_campaign:

  data = get_data(i, new_df, 'TFID')[0]
  index = get_data(i, new_df, 'TFID')[1]
  rows,data,rec_product_id,master_cam,prod_master  = get_TopN(index, data, cls)

  df_row = df_row.append(rows,ignore_index=True)
  df_product = df_product.append(data,ignore_index=True)
  df_master_id = df_master_id.append(master_cam,ignore_index=True)
  df_product_id = df_product_id.append(rec_product_id,ignore_index=True)
  df_master_product = df_master_product.append(prod_master,ignore_index=True)
  #print('\n')

df_row = df_row.rename(columns={0:'NumberOfRecommend'})  
df_product = df_product.rename(columns={0:'ProductNameRecommend'})
df_master_id = df_master_id.rename(columns={0:'MasterCampaignID'})
df_product_id = df_product_id.rename(columns={0:'CampaignIDRecommend'})
df_master_product = df_master_product.rename(columns={0:'MasterCampaignName'})
## Merge Result to Data Frame
result = pd.merge(df_row, df_master_id, left_index=True, right_index=True)\
          .merge(df_master_product,left_index=True, right_index=True)\
          .merge(df_product_id , left_index=True, right_index=True)\
          .merge(df_product,left_index=True, right_index=True).drop_duplicates()


## Choose Data Product Top 5
result2 = result.copy()
result2['Rank'] =  result2.groupby(['MasterCampaignID','MasterCampaignName']).cumcount()+1
result2 = result2[result2['Rank'] <= 5] # Choose 5 Product

In [0]:
## Result Bundle recommend by rank

display(result2)

In [0]:
actual_price = transaction_pd2[['campaignid','categoryname','price']].drop_duplicates()
result3 = result2.merge(actual_price.rename(columns={'price':'MasterPrice'}),left_on='MasterCampaignID',right_on='campaignid',how='left')\
        .merge(actual_price[['campaignid','price']].rename(columns={'campaignid':'Recommendcampaignid','price':'RecommendPrice'}),left_on='CampaignIDRecommend',right_on='Recommendcampaignid',how='left')

result3 = result3[['NumberOfRecommend','MasterCampaignID','MasterCampaignName','MasterPrice','CampaignIDRecommend','ProductNameRecommend','RecommendPrice','Rank']]
display(result3)

In [0]:
display(result3[result3['MasterCampaignID']== 11111])

In [0]:
display(result3[result3['MasterCampaignID']== 22222])

#10. Save Model And Result

##10.1 Save Model

In [0]:
from datetime import datetime,timedelta
import pickle


fileName = 'Model_%s_%s_%s.pkl' % (name,version,createdate)
path_txt = "/dbfs/mnt/blob/%s%s" % (location,fileName)
print(path_txt)

with open(path_txt, 'wb') as fp:
    pickle.dump(cls, fp)

/dbfs/mnt/blob/recommendation/Lays Thailand/Model_CampaignsrecommLays_V1_20221018.pkl

##10.2 Insert Data to table

In [0]:
import requests
import json

def createConnectionStringFromKeyVault(keyVaultAPI,secretKey,storageAccount):
  x = requests.get(f'{keyVaultAPI}&secretKey={secretKey}')
  sasKey = json.loads(x.text)['secret_value']
  
  connectionString = f'....'
  return connectionString

In [0]:
from azure.cosmosdb.table.tableservice import TableService,TableBatch
from azure.cosmosdb.table.models import EntityProperty,EdmType,Entity

def getSASKey(table_storage_name):
  x = requests.get(f'api={table_storage_name}')
  y = json.loads(x.text)
  return y['secret_value']

storage_account = 'account'
storage_sas_key = getSASKey(storage_account)
table_service = TableService(account_name=storage_account, sas_token = storage_sas_key)


In [0]:
tablename = 'CampaignsRecomm%s%s%s' % (modelversion,version,appid)
tablename

Out[68]: 'CampaignsRecommM1V1M1655874772'

In [0]:
result4 = result3[['MasterCampaignID','CampaignIDRecommend','MasterCampaignName','ProductNameRecommend','Rank']]

## Insert to azure blob

In [0]:
import numpy as np
import logging

for row in result4.iterrows():

  entity = {'PartitionKey': str(row[1]['MasterCampaignID']), 
            'RowKey': str(row[1]['CampaignIDRecommend']),
            'MasterCampaignName':str(row[1]['MasterCampaignName']),
            'ProductNameRecommend':row[1]['ProductNameRecommend'],
            'Rank':row[1]['Rank']
           }

  try:
    table_service.insert_entity(tablename,entity)
  except:
    table_service.update_entity(tablename,entity)
    pass